## Import all necessary modules

In [72]:
import math,os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

## Read data in "ExperimentalData" folder

In [116]:
# assign path and make a list of all files in directory
path, dirs, files = next(os.walk("./ExperimentalData/"))
file_count = len(files)
Files=[]

for i in range(len(dirs)):
    path, dis, files = next(os.walk("./ExperimentalData/"+dirs[i]))
    Files.append(files)
    file_count = file_count + len(files)

# create empty list
dataframes_list = []
 
# append datasets to the list (checks if it is CSV before doing so)
for k in range(len(dirs)):
    for i in range(len(Files[k])):
        if Files[k][i].endswith(".csv"):
            temp_df = pd.read_csv("./ExperimentalData/"+dirs[k]+"/"+Files[k][i])
            dataframes_list.append(temp_df)

# display datasets
for dataset in dataframes_list:
    display(dataset)

,55,33,18
0,6,68,3
1,9,6,432


,4521,45377,132746,12,5
0,1,6,7,2,3
1,4,124,3,3654,75


,Name,Sweep (T1),Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Date,24/2/2023,NaN,NaN,NaN
1,Time,7:14:22 AM,NaN,NaN,NaN
2,Instrument,FSC6 - 101132/006,NaN,NaN,NaN
3,Firmware Version,V2.22,NaN,NaN,NaN
4,Instrument Mode,Spectrum Analyzer,NaN,NaN,NaN
...,...,...,...,...,...
669,198736507.936508,-101.898582458496,-110.74503326416,NaN,NaN
670,199052380.952381,-101.287231445313,-110.527809143066,NaN,NaN
671,199368253.968254,-101.155311584473,-110.159370422363,NaN,NaN
672,199684126.984127,-100.019577026367,-109.725288391113,NaN,NaN


,1,2
0,456,6
1,7,9


In [1]:
params = {}
data = []
i,k,l=0,0,0
with open('./ExperimentalData/pop/OSA reading at 790 mA 1850 - 1950 nm (1835).CSV') as f:
    section = ''
    for line in f:
        if line.strip() == '"[TRACE DATA]"':
            section = 'data'
        elif section == 'data':
            data.append([float(x) for x in line.strip().split(',')])

df = pd.DataFrame(data, columns=['Wavelength', 'Intensity'])

FileNotFoundError: [Errno 2] No such file or directory: './ExperimentalData/pop/OSA reading at 790 mA 1850 - 1950 nm (1835).CSV'

In [184]:
display(data)

[[1850.0, -69.867],
 [1850.1001, -79.546],
 [1850.2002, -71.213],
 [1850.3003, -67.692],
 [1850.4004, -68.438],
 [1850.5005, -68.391],
 [1850.6006, -68.755],
 [1850.7007, -67.769],
 [1850.8008, -67.873],
 [1850.9009, -68.033],
 [1851.001, -67.716],
 [1851.1011, -67.998],
 [1851.2012, -67.917],
 [1851.3013, -68.154],
 [1851.4014, -68.04],
 [1851.5015, -68.818],
 [1851.6016, -67.386],
 [1851.7017, -67.577],
 [1851.8018, -68.549],
 [1851.9019, -68.396],
 [1852.002, -69.107],
 [1852.1021, -68.279],
 [1852.2022, -67.618],
 [1852.3023, -68.301],
 [1852.4024, -67.263],
 [1852.5025, -69.619],
 [1852.6026, -210.0],
 [1852.7027, -69.554],
 [1852.8028, -67.908],
 [1852.9029, -67.498],
 [1853.003, -66.395],
 [1853.1031, -67.569],
 [1853.2032, -67.222],
 [1853.3033, -67.988],
 [1853.4034, -67.835],
 [1853.5035, -70.124],
 [1853.6036, -89.742],
 [1853.7037, -70.356],
 [1853.8038, -67.923],
 [1853.9039, -67.825],
 [1854.004, -66.777],
 [1854.1041, -66.949],
 [1854.2042, -67.507],
 [1854.3043, -68.562

autocorrelator reading is readable but need to check for ".txt" 

RF reading (1000-o...) is readable but need cleaning

OSA reading is not readable because there are no commas in the first three lines

oscilloscope is no problemo!